In [108]:
import pandas as pd

# Falls noch nicht geladen:2025-10-22-qualification_data
df_priv = pd.read_csv("df_eval_private_2025-11-03.csv", low_memory=False)
df_pub = pd.read_csv("df_eval_public_2025-11-03.csv", low_memory=False)
df_history = pd.read_csv("df_history_2025-11-03.csv", low_memory=False)
df_eval = pd.read_csv("df_IDs_for_eval_2025-11-03.csv", low_memory=False)


# 1) Spalte robust in datetime konvertieren
#dt = pd.to_datetime(df["Auftragsende_SOLL"], errors="coerce")

In [109]:
#df_priv
#df_pub
#df_history
#df_eval

In [110]:
# --- Schritt 1: Zeitspalten konvertieren ---

time_cols = [
    "Auftragseingang",
    "Auftragsende_SOLL",
    "AFO_Start_SOLL",
    "AFO_Ende_SOLL",
    "AFO_Start_IST",
    "AFO_Ende_IST",
    "Auftragsende_IST"
]

# Falls einige Spalten fehlen (z.B. in eval datasets), ignorieren
existing_time_cols = [col for col in time_cols if col in df_history.columns]

for col in existing_time_cols:
    df_history[col] = pd.to_datetime(df_history[col], errors="coerce")


In [111]:
df_history[existing_time_cols].isna().sum()


Auftragseingang          0
Auftragsende_SOLL        0
AFO_Start_SOLL           0
AFO_Ende_SOLL            0
AFO_Start_IST            0
AFO_Ende_IST             0
Auftragsende_IST     58851
dtype: int64

In [114]:
#df_history

In [122]:
cutoff_date = pd.Timestamp("2014-01-02")  # 1.1.2014 + alles davor raus

existing_time_cols = [c for c in time_cols if c in df_history.columns]

# Maske: TRUE = Auftrag enthält alte Zeitstempel
mask_old = df_history[existing_time_cols].lt(cutoff_date).any(axis=1)

# Anzahl der betroffenen Zeilen
print("Anzahl Zeilen mit alten Datumswerten:", mask_old.sum())

# Aufträge entfernen
df_history = df_history.loc[~mask_old].copy()

Anzahl Zeilen mit alten Datumswerten: 302


In [154]:
45946
#df_history

45946

In [134]:
# --- Schritt: Eval-IDs aus der History entfernen ---

# Set der IDs, die vorhergesagt werden sollen
eval_ids = set(df_eval["AuftragsID"].unique())

# Maske: Zeilen, die zu Eval-IDs gehören
mask_eval = df_history["AuftragsID"].isin(eval_ids)

# Neue Dateien
df_history_clean = df_history.loc[~mask_eval].copy()   # History ohne Eval IDs
df_history_eval_removed = df_history.loc[mask_eval].copy()  # Nur die Eval-Aufträge

# Sortiert nach ID (absteigend)
df_history_eval_removed = df_history_eval_removed.sort_values(
    by="AuftragsID",
    ascending=False
).reset_index(drop=True)


print("Anzahl gelöschter Eval-Zeilen:", mask_eval.sum())
print("Neue History-Größe:", df_history_clean.shape)
print("Eval-Only-Größe:", df_history_eval_removed.shape)


Anzahl gelöschter Eval-Zeilen: 58828
Neue History-Größe: (1360869, 18)
Eval-Only-Größe: (58828, 18)


In [156]:
#df_history_eval_removed

In [146]:
# Eval Files kombinieren (je eine Zeile pro Auftrag)
df_eval_combined = pd.concat([df_pub, df_priv], axis=0, ignore_index=True)

df_eval_full = df_eval_combined.merge(
    df_history_eval_removed,
    on="AuftragsID",
    how="left",
    suffixes=("_eval", "_hist")
)


In [158]:
df_eval_full = df_history_eval_removed.sort_values(
    by="AuftragsID",
    ascending=False
).reset_index(drop=True)
#df_eval_full


In [150]:
# --- Export der bereinigten History (ohne Eval IDs) ---
df_history_clean.to_csv(
    "df_history_clean.csv",
    index=False,
    encoding="utf-8"
)


# --- Export der kombinierten Eval-Datei (pub + priv + history) ---
df_eval_full.to_csv(
    "df_eval_fulldetails.csv",
    index=False,
    encoding="utf-8"
)

print("Export abgeschlossen.")


Export abgeschlossen.


In [160]:
#df_history_clean

In [166]:
# Dummy-Datum
import numpy as np

DUMMY_DATE = "2024-1-12"

# Submission-DF erzeugen
df_solution = pd.DataFrame()
df_solution["AuftragsID"] = df_eval["AuftragsID"].astype(int)
df_solution["Auftragsende_PREDICTED"] = DUMMY_DATE

# ID-Spalte erzeugen (1...N)
df_solution["ID"] = np.arange(1, len(df_solution) + 1)

# Reihenfolge der Spalten korrekt setzen
df_solution = df_solution[["ID", "AuftragsID", "Auftragsende_PREDICTED"]]

# Export
df_solution.to_csv("test_submission.csv", index=False)

In [168]:
df_hist_bt1 = df_history_clean[df_history_clean["BauteilID"] == 1].copy()
df_hist_bt2 = df_history_clean[df_history_clean["BauteilID"] == 2].copy()
df_hist_bt3 = df_history_clean[df_history_clean["BauteilID"] == 3].copy()

print(df_hist_bt1.shape, df_hist_bt2.shape, df_hist_bt3.shape)


(550842, 18) (762709, 18) (47318, 18)


In [170]:
df_bt1_step1 = df_hist_bt1[df_hist_bt1["Arbeitsschritt"] == 1].copy()
df_bt2_step1 = df_hist_bt2[df_hist_bt2["Arbeitsschritt"] == 1].copy()
df_bt3_step1 = df_hist_bt3[df_hist_bt3["Arbeitsschritt"] == 1].copy()

print(df_bt1_step1.shape, df_bt2_step1.shape, df_bt3_step1.shape)


(59002, 18) (69284, 18) (6451, 18)


In [172]:
# --- Funktion: Dauer berechnen und Median zurückgeben ---
def berechne_median_prozessdauer(df):
    # Dauer in Stunden oder Tagen – du kannst wählen.
    # Ich nehme erstmal Stunden.
    df = df.copy()
    df["prozessdauer"] = (df["Auftragsende_IST"] - df["AFO_Start_IST"]).dt.total_seconds() / 3600
    return df["prozessdauer"].median()


# --- Median pro Bauteil berechnen ---
median_bt1 = berechne_median_prozessdauer(df_bt1_step1)
median_bt2 = berechne_median_prozessdauer(df_bt2_step1)
median_bt3 = berechne_median_prozessdauer(df_bt3_step1)

print("Median Prozessdauer Bauteil 1 (Stunden):", median_bt1)
print("Median Prozessdauer Bauteil 2 (Stunden):", median_bt2)
print("Median Prozessdauer Bauteil 3 (Stunden):", median_bt3)

# --- Gesamtmedian über alle Bauteile ---
df_all_step1 = pd.concat([df_bt1_step1, df_bt2_step1, df_bt3_step1], axis=0, ignore_index=True)
median_all = berechne_median_prozessdauer(df_all_step1)

print("Gesamtmedian (alle Bauteile):", median_all)


Median Prozessdauer Bauteil 1 (Stunden): 5500.708333333333
Median Prozessdauer Bauteil 2 (Stunden): 5598.616666666667
Median Prozessdauer Bauteil 3 (Stunden): 25.5
Gesamtmedian (alle Bauteile): 5426.85


In [178]:
# --- Median-Addition pro Bauteil auf df_eval_full ---

# Mapping der bauteilspezifischen Mediane
median_map = {
    1: median_bt1,
    2: median_bt2,
    3: median_bt3
}

# Nur Arbeitsschritt 1 aus df_eval_full

df_eval_step1 = df_eval_full[df_eval_full["Arbeitsschritt"] == 1].copy()

# Median zuordnen

df_eval_step1["median_hours"] = df_eval_step1["BauteilID"].map(median_map)

df_eval_step1["median_td"] = pd.to_timedelta(df_eval_step1["median_hours"], unit="h")

# Prediction berechnen: AFO_Start_IST + median

df_eval_step1["pred_end_datetime"] = (
    df_eval_step1["AFO_Start_IST"] + df_eval_step1["median_td"]
)

# Finale Ausgabe im Date-Format für Kaggle

df_eval_step1["Auftragsende_PREDICTED"] = df_eval_step1["pred_end_datetime"].dt.strftime("%Y-%m-%d")


In [180]:
df_eval_step1.head()


,AuftragsID,BauteilID,Bauteilbezeichnung,Auftragseingang,Priorität,Auftragsende_SOLL,Arbeitsschritt,Arbeitsschrittbezeichnung,AFO_Start_SOLL,AFO_Ende_SOLL,...,MaschinenID,Maschinenbezeichnung,Kapazität,Auftragsende_IST,AFO_Dauer_IST_Stunde,Auftragsdauer_IST_Tag,median_hours,median_td,pred_end_datetime,Auftragsende_PREDICTED
0,150368,3,Daempfungseinheit,2024-01-12,1,2024-03-01 11:02:00,1,Info,2024-03-01 07:00:00,2024-03-01 07:01:00,...,NaN,NaN,NaN,NaT,0.02,NaN,25.500000,1 days 01:30:00,2024-03-02 08:30:00.000000000,2024-03-02
3,150367,3,Daempfungseinheit,2023-12-17,1,2024-03-01 10:12:00,1,Info,2024-03-01 07:00:00,2024-03-01 07:01:00,...,NaN,NaN,NaN,NaT,0.02,NaN,25.500000,1 days 01:30:00,2024-03-02 08:30:00.000000000,2024-03-02
5,150366,3,Daempfungseinheit,2023-09-08,1,2024-03-01 10:12:00,1,Info,2024-03-01 07:00:00,2024-03-01 07:01:00,...,NaN,NaN,NaN,NaT,0.02,NaN,25.500000,1 days 01:30:00,2024-03-02 08:30:00.000000000,2024-03-02
7,150365,2,Schwenkzylinder,2023-09-26,1,2024-03-04 11:32:00,1,Info,2024-03-01 07:00:00,2024-03-01 07:01:00,...,NaN,NaN,NaN,NaT,0.02,NaN,5598.616667,233 days 06:37:00.000000001,2024-10-20 13:37:00.000000001,2024-10-20
9,150363,2,Schwenkzylinder,2023-10-25,1,2024-03-01 13:42:00,1,Info,2024-03-01 07:00:00,2024-03-01 07:01:00,...,NaN,NaN,NaN,NaT,0.02,NaN,5598.616667,233 days 06:37:00.000000001,2024-10-20 13:37:00.000000001,2024-10-20


In [182]:
# --- 1) Nur die benötigten Spalten auswählen ---
df_solution = df_eval_step1[["AuftragsID", "Auftragsende_PREDICTED"]].copy()

# --- 2) ID-Spalte erzeugen (1 bis N) ---
df_solution["ID"] = np.arange(1, len(df_solution) + 1)

# --- 3) Reihenfolge der Spalten sicherstellen ---
df_solution = df_solution[["ID", "AuftragsID", "Auftragsende_PREDICTED"]]

# --- 4) Exportieren ---
df_solution.to_csv("submission_median_model.csv", index=False, encoding="utf-8")

print("Fertige CSV exportiert: submission_median_model.csv")

Fertige CSV exportiert: submission_median_model.csv


In [184]:
df_solution.head()
df_solution.info()


<class 'pandas.core.frame.DataFrame'>
Index: 8530 entries, 0 to 58827
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      8530 non-null   int64 
 1   AuftragsID              8530 non-null   int64 
 2   Auftragsende_PREDICTED  8530 non-null   object
dtypes: int64(2), object(1)
memory usage: 266.6+ KB
